In [1]:
import io
import subprocess
import re
import pickle
import random

# TPTP Formatting
fof(name,formula_type,statement)
- name is the name of the formula
- formula_type is either axiom or conjecture, for our purposes.
- Axioms are facts
- Conjectures are what we want to prove/disprove


Extra TPTP notes:
- ! is the universal quantifier
- ? is the existential quantifier
- variables are denoted with capital letters
- func(C) is a function with variable C
- => is the implication operator
- & is AND
- | is OR
- ~ is NEGATION
- <=> is EQUIVALENCE
- = is EQUALITY
- != is INEQUALITY

In [2]:
class Axiom:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",axiom,"+str(self.statement)+")."

class Conjecture:
    def __init__(self,name,statement):
        self.name = name
        self.statement = statement
    def __str__(self):
        return "fof("+str(self.name)+",conjecture,"+str(self.statement)+")."

In [3]:
theorum = str(Axiom("1","b"))+str(Axiom("2","(a&b)=>b"))+str(Axiom("3","b=>c"))+str(Conjecture("c1","c"))

print(str(Axiom("1","a")))


fof(1,axiom,a).


In [4]:
#needs E, can't run on colab
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
with io.open(file_path,'w',encoding='utf-8') as f:
    f.write(theorum)

result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
output = result.stdout.decode()

if result.returncode == 0:
    #proof found
    print(output)
    pass

elif result.returncode == 1:
    #proof not found
    print(output)
    pass

else:
    #something else happened
    print(result)
    raise Exception("Something unexpected occured")

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_1, plain, (b)).
#
#cnf(i_0_3, plain, (c)).
#
# Proof found!
# SZS status Theorem
# SZS output start CNFRefutation
fof(c1, conjecture, c, file('f.tptp', c1)).
fof(3, axiom, (b=>c), file('f.tptp', 3)).
fof(1, axiom, b, file('f.tptp', 1)).
fof(c_0_3, negated_conjecture, ~c, inference(fof_simplification,[status(thm)],[inference(assume_negation,[status(cth)],[c1])])).
fof(c_0_4, plain, (~b|c), inference(fof_nnf,[status(thm)],[inference(fof_nnf,[status(thm)],[3])])).
fof(c_0_5, negated_conjecture, ~c, inference(fof_nnf,[status(thm)],[c_0_3])).
cnf(c_0_6, plain, (c|~b), inference(split_conjunct,[status(thm)],[c_0_4])).
cnf(c_0_7, plain, (b), inference(split_conjunct,[status(thm)],[1])).
cnf(c_0_8, negated_conjecture, (~c), inference(split_conjunct,[status(thm)],[c_0_5])).
cnf(c_0_9, plain, (c), inference(cn,[status(thm)],[inference(rw,[status(thm)],[c_0_6, c_0_7])])).
cnf(c_0_10, negated_conjecture, ($false), inference(cn,[status(

# Parse the proof output.
Todo: get rid of unnecessary info like filenames

In [5]:
parsed_result = ""
for line in output.split('\n'):
    if len(line) > 0 and line[0] == '#':
        pass
    else:
        line = line.replace(' ','')
        print(line)
        break
        

fof(c1,conjecture,c,file('f.tptp',c1)).


# Generate expressions using:
- and &
- or |
- not ~
- implies =>
- a1,a2,a3 etc for axiom names
- c1,c2,c3 etc for conjecture names


In [6]:
operation_list = ['&','|']

original = ['a','b','~a','~b',]
prop_names = []

for x in original:
    prop_names.append(str(x))

ops = operation_list
new = dict()
new2 = list()


for name in prop_names:
    new[str(name)]=str(name)

#i here is the length of the LHS of the axiom names, for example i=0 a=>a, i=1, a&a=>a, i=2 a&a|a=>a 
for i in range(1):
    for var1 in original:
        for var2 in prop_names:
            for o in ops:
                #may need regular expressions here to remove duplicated elements
                new[str(var1)+str(o)+str(var2)]=str(var1)+str(o)+str(var2)
        
    for e in new:
        original.append(str(new[e]))

for element in new:
    for name in prop_names:
        result = '('+str(new[element])+")=>"+str(name)
        new2.append(result)

In [7]:
#no need to run this on colab
print(len(new2))
with open('data.pickle','wb') as f:
    pickle.dump(new2,f)

with open('data.pickle','rb') as f:
    data = pickle.load(f)

for i in range(10):
    print(data[random.randint(0,(len(data))-1)])

144
(~b|~b)=>~a
(b&b)=>~b
(~b)=>a
(~b|~b)=>b
(~a)=>~b
(b|~a)=>~b
(~a&b)=>b
(~b|~b)=>~a
(b)=>b
(a)=>b


In [8]:
statements = new2
conclusions = prop_names

statement_list = list()
conclusion_list = list()


for i in range(len(statements)):
    statement_list.append(Axiom(str(i),statements[i]))
for i in range(len(conclusions)):
    conclusion_list.append(Conjecture(str(i),conclusions[i]))

theorum_list = list()
for s in statement_list:
    for c in conclusion_list:
        theorum_list.append(str(s)+str(c))


#needs E, can't run on colab
file_path = "f.tptp"
eprover_path = "/home/anmarch/source/eprover/PROVER/eprover"
found_proofs = list()
unfound_proofs = list()
for t in theorum_list:
    with io.open(file_path,'w',encoding='utf-8') as f:
        f.write(str(t))

    result = subprocess.run([eprover_path, "--proof-object", str(file_path)], capture_output=True)
    output = result.stdout.decode()

    if result.returncode == 0:
        #proof found
        #print(output)
        found_proofs.append([str(t),result])
        pass

    elif result.returncode == 1:
        #proof not found
        unfound_proofs.append([str(t),result])
        #print(output)
        pass

    else:
        #something else happened
        print(result)
        raise Exception("Something unexpected occured")

In [9]:
print(len(found_proofs))
print(len(unfound_proofs))
print(len(found_proofs)+len(unfound_proofs))
print(len(theorum_list))

44
532
576
576


# CNF creation
- & is AND
- | is OR
- ~ is NOT
- CNF -> (Disjunction) & CNF
- CNF -> (Disjunction)
- Disjunction -> Literal | Disjunction
- Disjunction -> Literal
- Literal -> ~Variable
- Literal -> Variable


In [10]:
variable_list = ['a','b','c']
literal_list = []

def produce_disjunctions(literal_list):
    disjunction_list = []
    disjunction_list_old = []

    for l in literal_list:
        disjunction_list_old.append(l)
        disjunction_list.append(l)

    for d in disjunction_list_old:
        for l in literal_list:
            disjunction_list.append(str(l)+'|'+str(d))

    return disjunction_list

def produce_cnf(disjunction_list):
    cnf_list_new = []
    cnf_list_old = []

    for d in disjunction_list:
        cnf_list_old.append('('+str(d)+')')
        cnf_list_new.append('('+str(d)+')')

    for c in cnf_list_old:
        for d in disjunction_list:
            cnf_list_new.append('('+str(d)+')&'+str(c))

    return cnf_list_new


for var in variable_list:
    for i in range(2):
        if i % 2 == 0:
            literal_list.append(str(var))
        else:
            literal_list.append('~'+str(var))

d_list = produce_disjunctions(literal_list)
cnf_list = produce_cnf(d_list)

d_list2 = produce_disjunctions(d_list)
cnf_list2 = produce_cnf(d_list2)

3263442
